In [142]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, DataFrames, JLD2, Dates

# Inputs

In [143]:
# Station information
stations_info = CSV.read("../../data/stations/station_information.csv", DataFrame);

#### for top 30 stations

In [144]:
# Capacity
C_30=CSV.read("../../data/parameters/top30/C.csv", DataFrame, header=false);
# initial bikes availability
y0_30_1=CSV.read("../../data/parameters/top30/y0_00.csv", DataFrame, header=false);
y0_30_2=CSV.read("../../data/parameters/top30/y0_55.csv", DataFrame, header=false);
y0_30_3=CSV.read("../../data/parameters/top30/y0_58.csv", DataFrame, header=false);
# Distances
D_30 = load("../../data/parameters/top30/D.jld2","D_30");
# Feasibility
X_30 = load("../../data/parameters/top30/X.jld2","X_30");
Xfeas_30 = load("../../data/parameters/top30/X_feasible.jld2","Xfeas_30");
# Demand
d_30_1 = load("../../data/parameters/top30/d_1.jld2","d_30_1");
d_30_2 = load("../../data/parameters/top30/d_2.jld2","d_30_2");
d_30_3 = load("../../data/parameters/top30/d_3.jld2","d_30_3");

### for top 50 stations

In [145]:
# Capacity
C_50=CSV.read("../../data/parameters/top50/C.csv", DataFrame, header=false);
# initial bikes availability
y0_50_1=CSV.read("../../data/parameters/top50/y0_00.csv", DataFrame, header=false);
y0_50_2=CSV.read("../../data/parameters/top50/y0_55.csv", DataFrame, header=false);
y0_50_3=CSV.read("../../data/parameters/top50/y0_58.csv", DataFrame, header=false);
# Distances
D_50 = load("../../data/parameters/top50/D.jld2","D_50");
# Feasibility
X_50 = load("../../data/parameters/top50/X.jld2","X_50");
Xfeas_50 = load("../../data/parameters/top50/X_feasible.jld2","Xfeas_50");
# Demand
d_50_1 = load("../../data/parameters/top50/d_1.jld2","d_50_1");
d_50_2 = load("../../data/parameters/top50/d_2.jld2","d_50_2");
d_50_3 = load("../../data/parameters/top50/d_3.jld2","d_50_3");

# Model - Sequential

In [146]:
function solve_sequential_model(K,S,C,D,X,y0,d,lambda,time_limit,save)
    model = Model(Gurobi.Optimizer)
    # set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", time_limit)

    n_stations, _, n_hours = size(d)
    C=C[!,1]
    y0=y0[!,1]
    # Decision variables
    @variable(model, x[1:n_stations, 1:n_stations, 1:K, 1:n_hours], Bin)
    @variable(model, v[1:n_stations, 1:n_stations, 1:K, 1:n_hours], Bin)
    @variable(model, 0 <= z[1:n_stations, 1:n_stations, 1:K, 1:n_hours], Int)
    @variable(model, 0 <= y[1:n_stations, 1:n_hours]) # Int by definition of the constraints
    @variable(model, 0 <= w[1:n_stations, 1:n_stations, 1:n_hours], Int)
    @variable(model, 0 <= u[1:n_stations, 1:n_stations, 1:n_hours])

    # Add constraints:
    # Stations capacity
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], y[i,t] <= C[i])
    # Flow balance 
    @constraint(model, [i in 1:n_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations)-sum(z[i,j,k,t] for j in 1:n_stations, k in 1:K)+sum(z[j,i,k,t] for j in 1:n_stations, k in 1:K))
    # Flow balance for the first hour 
    @constraint(model, [i in 1:n_stations], y[i,1] - y0[i] == sum(w[j,i,1] for j in 1:n_stations)-sum(w[i,j,1] for j in 1:n_stations)-sum(z[i,j,k,1] for j in 1:n_stations, k in 1:K)+sum(z[j,i,k,1] for j in 1:n_stations, k in 1:K))
    # Vans capacity
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= S)
    # Users travel availability
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], -y[i,t] <= sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations)<= C[i] - y[i,t])
    
    # Max 1 rebalancing per hour per van
    @constraint(model, [k in 1:K, t in 1:n_hours], sum(x[i,j,k,t] for i in 1:n_stations, j in 1:n_stations) == 1)
    @constraint(model, [k in 1:K, t in 1:n_hours], sum(v[i,j,k,t] for i in 1:n_stations, j in 1:n_stations) == 1)
    # Rebalancing feasibility
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= S*x[i,j,k,t])
    for i in 1:n_stations
        for j in 1:n_stations
            if X[i,j] == 0
                for k in 1:K
                    for t in 1:n_hours
                        @constraint(model, x[i,j,k,t] == 0)
                        @constraint(model, v[i,j,k,t] == 0)
                        @constraint(model, z[i,j,k,t] == 0)
                    end
                end
            end
        end
    end
   # Vans must travel sequentially
    @constraint(model, [i in 1:n_stations, k in 1:K, t in 2:n_hours], sum(x[i,j,k,t] for j in 1:n_stations) <= sum(v[l,i,k,t-1] for l in 1:n_stations))
    @constraint(model, [j in 1:n_stations, k in 1:K, t in 1:n_hours], sum(v[j,l,k,t] for l in 1:n_stations) <= sum(x[i,j,k,t] for i in 1:n_stations))
    
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], w[i,j,t] <= d[i,j,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], u[i,j,t] >= d[i,j,t] - w[i,j,t])
    
    # Users travel is inferior to the demand and definition of u
    #for eps in -2:1:2 # for Robustness
    #    for i in 1:n_stations
    #        for j in 1:n_stations
    #            for t in 1:n_hours
    #                if d[i,j,t] + eps >=0
    #                    @constraint(model, u[i,j,t] >= d[i,j,t] + eps - w[i,j,t])
    #                    @constraint(model, d[i,j,t] + eps >= w[i,j,t])
    #                end
    #            end
    #        end
    #    end    
    #end

    # Set objective
    @objective(model, Min, sum(u[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:n_hours)+lambda*sum(D[i,j]*(x[i,j,k,t]+v[i,j,k,t]/3) for i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:n_hours))
    
    # Solve the model
    optimize!(model)
    
    # Export the solution
    x_sol = value.(x)
    v_sol = value.(v)
    z_sol = value.(z)
    w_sol = value.(w)
    y_sol = value.(y)
    u_sol = value.(u)

    if save
        # current date without millisecond and no colon
        current_date = Dates.format(now(), "yyyy-mm-ddTHHMMSS")
        folder = "../../results/data/"*string(current_date)*"_nstations_"*string(n_stations)*"_nvans_"*string(K)*"_capacity_"*string(S)*"_lambda_"*string(lambda)*"_timelimit_"*string(time_limit)*"/"
        mkdir(folder)
        CSV.write(folder*"x.csv", Tables.table(reshape(x_sol,:)), writeheader=false)
        CSV.write(folder*"v.csv", Tables.table(reshape(v_sol,:)), writeheader=false)
        CSV.write(folder*"z.csv", Tables.table(reshape(z_sol,:)), writeheader=false)
        CSV.write(folder*"w.csv", Tables.table(reshape(w_sol,:)), writeheader=false)
        CSV.write(folder*"y.csv", Tables.table(reshape(y_sol,:)), writeheader=false)
        CSV.write(folder*"u.csv", Tables.table(reshape(u_sol,:)), writeheader=false)
        CSV.write(folder*"d.csv", Tables.table(reshape(d,:)), writeheader=false)
    end

    return value.(x), value.(v), value.(w), value.(u), value.(y), value.(z), objective_value(model)
end

solve_sequential_model (generic function with 1 method)

## No rebalancing model

In [147]:
function solve_no_rebalancing_model(C,D,X,y0,d,time_limit,save)
    model = Model(Gurobi.Optimizer)
    # set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", time_limit)

    n_stations, _, n_hours = size(d)
    C=C[!,1]
    y0=y0[!,1]
    # Decision variables
    @variable(model, 0 == x[1:n_stations, 1:n_stations, 1:K, 1:n_hours])
    @variable(model, 0 == v[1:n_stations, 1:n_stations, 1:K, 1:n_hours])
    @variable(model, 0 == z[1:n_stations, 1:n_stations, 1:K, 1:n_hours])
    @variable(model, 0 <= y[1:n_stations, 1:n_hours]) # Int by definition of the constraints
    @variable(model, 0 <= w[1:n_stations, 1:n_stations, 1:n_hours], Int)
    @variable(model, 0 <= u[1:n_stations, 1:n_stations, 1:n_hours])

    # Add constraints:
    # Stations capacity
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], y[i,t] <= C[i])
    # Flow balance 
    @constraint(model, [i in 1:n_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations))
    # Flow balance for the first hour 
    @constraint(model, [i in 1:n_stations], y[i,1] - y0[i] == sum(w[j,i,1] for j in 1:n_stations)-sum(w[i,j,1] for j in 1:n_stations))
    # Users travel availability
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], -y[i,t] <= sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations)<= C[i] - y[i,t])
    
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], w[i,j,t] <= d[i,j,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], u[i,j,t] >= d[i,j,t] - w[i,j,t])
    # Users travel is inferior to the demand and definition of u
    #for eps in -2:1:2 # for Robustness
    #    for i in 1:n_stations
    #        for j in 1:n_stations
    #            for t in 1:n_hours
    #                if d[i,j,t] + eps >=0 
    #                    @constraint(model, u[i,j,t] >= d[i,j,t] + eps - w[i,j,t])
    #                    @constraint(model, d[i,j,t] + eps >= w[i,j,t])
    #                end
    #            end
    #        end
    #    end    
    #end

    # Set objective
    @objective(model, Min, sum(u[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:n_hours))
    
    # Solve the model
    optimize!(model)
    
    # Export the solution
    x_sol = value.(x)
    v_sol = value.(v)
    z_sol = value.(z)
    w_sol = value.(w)
    y_sol = value.(y)
    u_sol = value.(u)

    if save
        # current date without millisecond and no colon
        current_date = Dates.format(now(), "yyyy-mm-ddTHHMMSS")
        folder = "../../results/data/"*string(current_date)*"_nstations_"*string(n_stations)*"_nvans_"*string(K)*"_capacity_"*string(S)*"_lambda_"*string(lambda)*"_timelimit_"*string(time_limit)*"_norebal"*"/"
        mkdir(folder)
        CSV.write(folder*"x.csv", Tables.table(reshape(x_sol,:)), writeheader=false)
        CSV.write(folder*"v.csv", Tables.table(reshape(v_sol,:)), writeheader=false)
        CSV.write(folder*"z.csv", Tables.table(reshape(z_sol,:)), writeheader=false)
        CSV.write(folder*"w.csv", Tables.table(reshape(w_sol,:)), writeheader=false)
        CSV.write(folder*"y.csv", Tables.table(reshape(y_sol,:)), writeheader=false)
        CSV.write(folder*"u.csv", Tables.table(reshape(u_sol,:)), writeheader=false)
        CSV.write(folder*"d.csv", Tables.table(reshape(d,:)), writeheader=false)
    end

    return value.(x), value.(v), value.(w), value.(u), value.(y), value.(z), objective_value(model)
end

solve_no_rebalancing_model (generic function with 1 method)

### Parameters

In [148]:
# Number of vans
K=5
# Capacity of vans
S=20
# Trade-off for multi-objective
lambda=4; # to be tuned according to the objectives scales

## Solve for Day 1

### with top 30 stations

In [138]:
x_1,v_1,w_1,u_1,y_1,z_1,obj_1=solve_sequential_model(K,S,C_50,D_50,Xfeas_50,y0_50_1,d_50_1,lambda,300,true);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 802569 rows, 1062432 columns and 3991971 nonzeros
Model fingerprint: 0x8d1044ea
Variable types: 63648 continuous, 998784 integer (624240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [9e-02, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 493753 rows and 176230 columns
Presolve time: 3.22s
Presolved: 308816 rows, 886202 columns, 2946796 nonzeros
Variable types: 0 continuous, 886202 integer (589548 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.07s

Barrier statistics:
 AA' N

LoadError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [ ]:
findall(x -> x >0.5, z_1[:,:,:,:]);

In [ ]:
findall(x -> x >0.5, x_1[:,:,:,:]);

In [ ]:
# Count the number of non-moving trips
non_moving=sum(sum(x_1[i,i,:,:] for i in 1:30))
# Count the number of rebalancing trips
rebalancing=sum(sum(x_1[i,j,:,:] for i in 1:30, j in 1:30))-non_moving

19.999999966486996

In [107]:
#using Plots
#plot([(sum(sum(x_1[i,j,:,t] for i in 1:30, j in 1:30))-sum(sum(x_1[i,i,:,t] for i in 1:30))) for t=1:24], label="Rebalancing", xlabel="Hour", ylabel="Number of rebalancing", title="# rebalancing per hour on Day 1")

## Try different lambdas

--> lambda=4

In [106]:
#K=5
#S=25
#for lambda in 0.5:2.5:15.5
#    n_stations,_,n_hours=size(d_30_1)
#    x_1,v_1,w_1,u_1,y_1,z_1,obj_1=solve_sequential_model(K,S,C_30,D_30,Xfeas_30,y0_30_1,d_30_1,lambda,80,false);
#    nb_rebalancings=sum(sum(x_1[i,j,:,:] for i in 1:n_stations, j in 1:n_stations))-sum(sum(x_1[i,i,:,:] for i in 1:n_stations))
#    unmet_demand=sum(u_1[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:n_hours)
#    println("lambda = ", lambda, " nb rebalancings = ", nb_rebalancings, " unmet demand = ", unmet_demand)
#end

## Try different K

### for 30 stations

In [134]:
lambda=4
S=20
nb_rebalancings=zeros(10)
unmet_demand=zeros(10)
distance_vans=zeros(10)
# Build dict ou df avec results
for K in 1:10
    n_stations,_,n_hours=size(d_30_1)
    x_1,v_1,w_1,u_1,y_1,z_1,obj_1=solve_sequential_model(K,S,C_30,D_30,Xfeas_30,y0_30_1,d_30_1,lambda,160,false);
    nb_rebalancings[K]=sum(sum(x_1[i,j,:,:] for i in 1:n_stations, j in 1:n_stations))-sum(sum(x_1[i,i,:,:] for i in 1:n_stations))
    distance_vans[K]=sum(D_30[i,j]*(x_1[i,j,k,t]+v_1[i,j,k,t]) for i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:n_hours)
    unmet_demand[K]=sum(u_1[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:n_hours)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter TimeLimit to value 160
Set parameter TimeLimit to value 160
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 101129 rows, 116064 columns and 461487 nonzeros
Model fingerprint: 0xa7beaf8e
Variable types: 23808 continuous, 92256 integer (46128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-01, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 76977 rows and 49819 columns
Presolve time: 0.27s
Presolved: 24152 rows, 66245 columns, 222891 nonzeros
Variable types: 0 continuous, 66245 integer (43340 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.050392e+03, 3719

In [ ]:
analysis_K_30=Dict()
analysis_K_30["K"]=1:10
analysis_K_30["nb_rebalancings"]=nb_rebalancings
analysis_K_30["unmet_demand"]=unmet_demand
analysis_K_30["distance_vans"]=distance_vans
CSV.write("../../results/data/analysis_K_30.csv", DataFrame(analysis_K_30))

"../../results/data/analysis_K_30.csv"

## Comparison with and without rebalancing model

In [140]:
K=5
lambda=4
S=20
x,v,w,u,y,z,obj=solve_sequential_model(K,S,C_30,D_30,Xfeas_30,y0_30_1,d_30_1,lambda,160,true);
x_,v_,w_,u_,y_,z_,obj_=solve_no_rebalancing_model(C_30,D_30,Xfeas_30,y0_30_1,d_30_1,160,true);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter TimeLimit to value 160
Set parameter TimeLimit to value 160
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 309229 rows, 392832 columns and 1480231 nonzeros
Model fingerprint: 0x7f1805b3
Variable types: 23808 continuous, 369024 integer (230640 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-01, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 195724 rows and 73746 columns
Presolve time: 1.12s
Presolved: 113505 rows, 319086 columns, 1061215 nonzeros
Variable types: 0 continuous, 319086 integer (212659 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   32